# categorize_transactions.py (notebook)

This notebook is a cell-by-cell conversion of the script `categorize_transactions.py`.
Run the code cells in order. Cells contain the same functions and logic as the original script, plus a small example showing how to run the `main` workflow from inside the notebook.

In [ ]:
import argparse
import csv
import os
import re
from collections import Counter, defaultdict
from typing import Dict, Tuple, Optional, List

import pandas as pd

# Try rapidfuzz first; if unavailable, fall back to difflib
try:
    from rapidfuzz import process, fuzz  # type: ignore
    HAVE_RAPIDFUZZ = True
except Exception:
    import difflib
    HAVE_RAPIDFUZZ = False


In [ ]:
PUNCT_RE = re.compile(r"[^\w\s]")
SPACE_RE = re.compile(r"\s+")

def normalize(text: str) -> str:
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    t = text.strip().lower()
    t = PUNCT_RE.sub(" ", t)         # remove punctuation
    t = SPACE_RE.sub(" ", t)         # collapse spaces
    return t.strip()

def most_common_pair(pairs: List[Tuple[str, str]]) -> Tuple[Optional[str], Optional[str]]:
    if not pairs:
        return None, None
    counter = Counter(pairs)
    (cat, subcat), _ = counter.most_common(1)[0]
    return cat, subcat

def build_description_lookup(dest_df: pd.DataFrame) -> Dict[str, Tuple[Optional[str], Optional[str]]]:
    # Group by normalized description and take the most common (category, sub-category) pair
    pairs_by_desc: Dict[str, List[Tuple[str, str]]] = defaultdict(list)
    for _, row in dest_df.iterrows():
        nd = normalize(row.get("description", ""))
        cat = row.get("category")
        sub = row.get("sub-category") if "sub-category" in row else row.get("sub_category")
        if pd.notna(cat) and pd.notna(sub):
            pairs_by_desc[nd].append((str(cat), str(sub)))
    lookup: Dict[str, Tuple[Optional[str], Optional[str]]] = {}
    for nd, pairs in pairs_by_desc.items():
        lookup[nd] = most_common_pair(pairs)
    return lookup

def fuzzy_best_match(query: str, choices: List[str], threshold: int = 90) -> Tuple[Optional[str], float]:
    if not choices:
        return None, 0.0
    if HAVE_RAPIDFUZZ:
        # Using token_set_ratio helps with word order/duplication
        match = process.extractOne(query, choices, scorer=fuzz.token_set_ratio)
        if match is None:
            return None, 0.0
        choice, score, _ = match
        return choice, float(score)
    else:
        # difflib returns [best, ...]; similarity 0..1
        best = None
        best_score = 0.0
        for c in choices:
            s = difflib.SequenceMatcher(None, query, c).ratio()
            if s > best_score:
                best_score = s
                best = c
        # Map 0..1 to 0..100 to align with threshold
        return best, best_score * 100.0


In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Categorize transactions by matching descriptions to a destination file (exact, then fuzzy).")
    parser.add_argument("--source", required=True, help="Path to source CSV (Date, description, amount, card)")
    parser.add_argument("--destination", required=True, help="Path to destination CSV (Date, description, amount, card, category, sub-category)")
    parser.add_argument("--threshold", type=int, default=90, help="Fuzzy match threshold (0-100). Default: 90")
    parser.add_argument("--reviewed", default=None, help="Optional path to a reviewed CSV (from to_review.csv) to apply final categories before writing updated_destination.csv")
    parser.add_argument("--output_dir", default=".", help="Directory to write outputs")
    args = parser.parse_args()

    os.makedirs(args.output_dir, exist_ok=True)

    # Read CSVs (case-insensitive column handling)
    def standardize_cols(df: pd.DataFrame) -> pd.DataFrame:
        lower_map = {c: c.strip().lower() for c in df.columns}
        df = df.rename(columns=lower_map)
        # normalize common variants
        if "sub_category" in df.columns and "sub-category" not in df.columns:
            df = df.rename(columns={"sub_category": "sub-category"})
        return df

    src = standardize_cols(pd.read_csv(args.source))
    dst = standardize_cols(pd.read_csv(args.destination))

    required_src = {"date", "description", "amount", "card"}
    if not required_src.issubset(set(src.columns)):
        raise ValueError(f"Source file must contain columns {required_src}. Found: {list(src.columns)}")

    required_dst = {"date", "description", "amount", "card", "category", "sub-category"}
    if not required_dst.issubset(set(dst.columns)):
        missing = required_dst - set(dst.columns)
        raise ValueError(f"Destination file missing required columns: {missing}")

    # Build lookup for exact description -> (category, subcat)
    desc_to_pair = build_description_lookup(dst)
    all_choices = list(desc_to_pair.keys())

    # Prepare output rows
    suggestions = []
    for _, row in src.iterrows():
        nd = normalize(row["description"])
        cat = None
        sub = None
        match_type = "none"
        match_desc = None
        match_score = 0.0

        # Exact
        if nd in desc_to_pair and all(v is not None for v in desc_to_pair[nd]):
            cat, sub = desc_to_pair[nd]
            match_type = "exact"
            match_desc = nd
            match_score = 100.0
        else:
            # Fuzzy
            best, score = fuzzy_best_match(nd, all_choices, threshold=args.threshold)
            if best is not None and score >= args.threshold and all(v is not None for v in desc_to_pair.get(best, (None, None))):
                cat, sub = desc_to_pair[best]
                match_type = "fuzzy"
                match_desc = best
                match_score = score

        suggestion = {
            "date": row.get("date"),
            "description": row.get("description"),
            "amount": row.get("amount"),
            "card": row.get("card"),
            "category": cat,
            "sub-category": sub,
            "match_type": match_type,
            "matched_description_norm": match_desc,
            "match_score": round(match_score, 1),
            "needs_review": match_type != "exact",  # review fuzzy and none
            "reason": "exact" if match_type == "exact" else ("fuzzy_suggest" if match_type == "fuzzy" else "no_match"),
        }
        suggestions.append(suggestion)

    suggest_df = pd.DataFrame(suggestions)

    # Save all rows with suggestions
    merged_path = os.path.join(args.output_dir, "merged_with_suggestions.csv")
    suggest_df.to_csv(merged_path, index=False)

    # Save rows needing review
    to_review = suggest_df[suggest_df["needs_review"]].copy()
    # Provide empty columns for user to fill/override
    to_review["category_final"] = to_review["category"]
    to_review["sub-category_final"] = to_review["sub-category"]
    review_path = os.path.join(args.output_dir, "to_review.csv")
    to_review.to_csv(review_path, index=False)

    print(f"Wrote: {merged_path}")
    print(f"Wrote: {review_path}")

    # If user provided a reviewed file, apply it and write updated_destination.csv
    if args.reviewed:
        reviewed = standardize_cols(pd.read_csv(args.reviewed))
        # Expect category_final and sub-category_final
        for col in ["category_final", "sub-category_final"]:
            if col not in reviewed.columns:
                raise ValueError(f"Reviewed file must include '{col}' column.")

        # Join reviewed choices back to suggestions on (date, description, amount, card)
        join_keys = ["date", "description", "amount", "card"]
        merged = suggest_df.merge(
            reviewed[join_keys + ["category_final", "sub-category_final"]],
            on=join_keys,
            how="left",
            suffixes=("", "_rev"),
        )
        # Prefer reviewed values when provided (not NaN)
        def coalesce(a, b):
            return b if pd.notna(b) and str(b).strip() != "" else a

        merged["category"] = [coalesce(a, b) for a, b in zip(merged["category"], merged["category_final"]) ]
        merged["sub-category"] = [coalesce(a, b) for a, b in zip(merged["sub-category"], merged["sub-category_final"]) ]

        # Build the updated destination by appending newly categorized rows from source.
        # Avoid duplicates: define a transaction id by (date, description, amount, card).
        dst_keys = ["date", "description", "amount", "card", "category", "sub-category"]
        # Source rows now with filled categories
        finalized_src = merged.copy()
        # Keep only required columns for destination
        finalized_src = finalized_src[dst_keys]

        # Combine with existing destination and drop duplicates
        updated_dst = pd.concat([dst[dst_keys], finalized_src], ignore_index=True)
        updated_dst.drop_duplicates(subset=["date", "description", "amount", "card"], keep="last", inplace=True)

        out_path = os.path.join(args.output_dir, "updated_destination.csv")
        updated_dst.to_csv(out_path, index=False)
        print(f"Wrote: {out_path}")


In [ ]:
if __name__ == "__main__":
    main()


## How to run in this notebook

You can either run the notebook cells sequentially and then run the script-like entry point, or call `main()` from a cell after setting `sys.argv`.
Example: set the arguments and call `main()` (the project includes `source.csv` and `destination.csv` in the workspace root):

In [ ]:
# Example: run the main workflow from within the notebook
import sys
# Adjust the paths below if your files are elsewhere
sys.argv = [
    'notebook',
    '--source', 'source.csv',
    '--destination', 'destination.csv',
    '--output_dir', '.',
]
# Call main() to execute the workflow (reads/writes CSVs)
main()


---
Created notebook `categorize_transactions.ipynb` mirroring the script.
Notes:
- If `rapidfuzz` is installed it will be used for fuzzy matching; otherwise the notebook falls back to `difflib`.
- Running the example cell will execute `main()` and produce `merged_with_suggestions.csv` and `to_review.csv` in the current directory.
- If you prefer not to run the entire script inside the notebook, you can import and call individual functions (e.g., `build_description_lookup`) for interactive exploration.